In [1]:
import pandas as pd
import numpy as np

from source.Files_operating import read_sht_df
from source.Plotting import plot_shoot
from source.Signal_processing import dbs_A_dFi, get_shoot_slices

shtRipper v1.3


In [2]:
# Main params

F_ID = 41226  # NUM (ID) OF CURRENT SHOOT

# Paths
proj_path = "C:/Users/f.belous/Work/Projects/Plasma_analysis"  # GLOBAL PATH TO PROJECT DIR - EDIT ONES
sht_dir_path = proj_path + "/data/sht/all/"  # GLOBAL PATH TO SHOOTS SHT DIR - EDIT ONES
dbs_dir_path = proj_path + "/data/dbs/sht/"  # GLOBAL PATH TO Dref SHT FILES DIR - EDIT ONES

# Filter & smoothing params
w = 0.8  # recent filtering param (better don't touch)
smooth_length = 200  # DBS points ( smooth_length / 4 = timedelta (msc) )
plot_smoothing = 100  # Smoothing points on plot

In [3]:
# Loading shoot data from SHT file in "sht_dir_path"

df = read_sht_df(f'sht{F_ID}', sht_dir_path,
                 column_names=[  #  SHT NAMES OF CHANNELS
                     "D-alfa  хорда R=50 cm",
                     "SXR 50 mkm",
                     "nl 42 cm (1.5мм) 64pi",
                     "МГД быстрый зонд верт.",
                     "МГД быстрый зонд рад.",
                 ],
                 data_names=[  # NAMES IN DF (eng & better to make it short & lowercase)
                    "d_alpha",
                     "sxr",
                     "nl",
                     "mgd_v",
                     "mgd_r",
                 ])

df.describe()

,t,d_alpha,sxr,nl,mgd_v,mgd_r
count,393216.000000,393216.000000,393216.000000,393216.000000,393216.000000,393216.000000
mean,0.196608,0.509547,0.268817,25.078660,-0.002498,-0.005745
std,0.113512,0.693328,0.464351,22.321558,0.417378,0.359830
min,0.000000,-0.069016,-0.002556,-10.843198,-5.235000,-5.235000
25%,0.098304,0.015337,0.081797,5.546851,-0.007668,-0.007668
50%,0.196607,0.048567,0.084353,16.962627,-0.002556,-0.005112
75%,0.294911,1.114482,0.143145,43.623296,0.002556,-0.002556
max,0.393215,4.872026,2.402783,73.254214,5.232444,5.043289


In [4]:
# Loading DBS data from "dbs_dir_path"
# Check existing of Dref file for this shoot!

channels = [3, 4, 5, 6, 7, 8]  # 1, 2, 3, 4, 5, 6, 7, 8 | DBS channels - to get frequency get journal of this shoot for DBS 
# (old: 1,2 - 39 GHz, ...; new: 1,2 - 20 GHz)
dbs_df = read_sht_df(f'Dref{F_ID}', dbs_dir_path,
                     column_names=[f"ch{channel}" for channel in channels])

dbs_df.describe()

,t,ch3,ch4,ch5,ch6,ch7,ch8
count,512000.000000,512000.000000,512000.000000,512000.000000,512000.000000,512000.000000,512000.000000
mean,0.164000,0.143689,0.128637,0.162208,0.141429,0.148554,0.142341
std,0.036951,0.105739,0.066848,0.248256,0.230570,0.912071,0.841359
min,0.100000,-0.546875,-0.330078,-1.310547,-1.669922,-4.000000,-3.429688
25%,0.132000,0.109375,0.105469,0.078125,0.068359,-0.289062,-0.283203
50%,0.164000,0.142578,0.126953,0.162109,0.144531,0.150391,0.148437
75%,0.196000,0.173828,0.148438,0.246094,0.224609,0.566406,0.515625
max,0.228000,1.113281,0.714844,2.041016,1.484375,3.998047,3.998047


In [5]:
# Gets A & dFi
# Check filter param - "w" & smoothing param - "smooth_length"!
dbs_A_dFi(dbs_df, w, smooth_length)

dbs_df.describe()

[3, 4, 5, 6, 7, 8]


,t,ch3,ch4,ch5,ch6,ch7,ch8,ch3_A,ch3_dfi,ch5_A,ch5_dfi,ch7_A,ch7_dfi
count,512000.000000,512000.000000,512000.000000,512000.000000,512000.000000,512000.000000,512000.000000,512000.000000,512000.000000,512000.000000,512000.000000,512000.000000,512000.000000
mean,0.164000,0.001126,0.001240,0.000764,0.000731,0.000225,0.000233,0.001825,-0.040509,0.001673,-0.139519,0.001563,0.089685
std,0.036951,0.000828,0.000644,0.001170,0.001191,0.001379,0.001378,0.000235,0.096496,0.000618,0.188261,0.000807,0.130249
min,0.100000,-0.004284,-0.003182,-0.006176,-0.008628,-0.006049,-0.005617,0.000781,-0.674505,0.000541,-0.911556,0.000104,-0.437608
25%,0.132000,0.000857,0.001017,0.000368,0.000353,-0.000437,-0.000464,0.001663,-0.032299,0.001130,-0.265121,0.000883,-0.000020
50%,0.164000,0.001117,0.001224,0.000764,0.000747,0.000227,0.000243,0.001776,-0.000563,0.001484,-0.057809,0.001575,0.067958
75%,0.196000,0.001362,0.001431,0.001160,0.001160,0.000857,0.000844,0.001963,0.000604,0.002155,-0.000004,0.002177,0.174442
max,0.228000,0.008721,0.006891,0.009619,0.007669,0.006046,0.006548,0.003145,0.485176,0.004015,0.440358,0.004570,0.645544


In [ ]:
# Gets slices by SXR fall & split first slice (before first SXR fall)

res_slices_edges = get_shoot_slices(df.d_alpha, df.sxr)

---

In [1]:
# Lists for saving data
# DO NOT RUN WHILE MARKING ONE SHOOT -- CLEAR ALL CURRENT DATA

df_groups_arr = []  # l_edge, r_edge, n, fr, fr_std
df_points_arr = []

In [ ]:
# Plotting cell
# You can set start index or input number of wanted slices (check twice)
start_index = 0
plot_shoot(F_ID, res_slices_edges, start_index, 2000,
           df, dbs_df, plot_smoothing, df_groups_arr, df_points_arr)

---
---

In [ ]:
# Stats of current data

print(f"Shot {F_ID}\n----")
print(f"Samples (total: {len(df_points_arr)}): EHO - {np.count_nonzero(np.array(df_points_arr)[:, 2] == 'eho')}, LCO - {np.count_nonzero(np.array(df_points_arr)[:, 2] == 'lco')}, dELM - {np.count_nonzero(np.array(df_points_arr)[:, 2] == 'delm')}")
print(f"Groups (total: {len(df_groups_arr)}): EHO - {np.count_nonzero(np.array(df_groups_arr)[:, -1] == 'eho')}, LCO - {np.count_nonzero(np.array(df_groups_arr)[:, -1] == 'lco')}, dELM - {np.count_nonzero(np.array(df_groups_arr)[:, -1] == 'delm')}")

In [ ]:
# SAVE CURRENT DATA

# l_edge, r_edge, n, fr, fr std
gr_df = pd.DataFrame(sorted(df_groups_arr, key=lambda x: x[0]), columns=["l_edge", "r_edge", "n", "fr", "fr_std", "mark"])
p_df = pd.DataFrame(sorted(df_points_arr, key=lambda x: x[0]), columns=["timepoint, ms", "fr, kHz", "mark"])

gr_df.to_csv(f"data/df/stats/groups/{F_ID}_groups_stats.csv", index=False)
p_df.to_csv(f"data/df/stats/samples/{F_ID}_points_dataset.csv", index=False)

# DO NOT FORGET TO CLEAR DATA FROM ARRAYS "df_groups_arr" "df_points_arr" BEFORE MARKING NEXT SHOOT

---